# Auxiliar 1: Encontrar identificadores com maior frequência de valores preenchidos para o Brasil, entre 1998 a 2013.

## 1. Importando libs

In [98]:
import pandas as pd
import zipfile
import numpy as np

## 2. Carregando o dataframe com todos os dados de todos os países e da faixa temporal de anos.

In [99]:
zf = zipfile.ZipFile('../data/Edstats_csv.zip')
text_files = zf.infolist()
df_todos_paises = pd.DataFrame()
for csv_file in text_files:        
    if csv_file.filename == 'EdStatsData.csv':
        print("Abrindo o arquivo",csv_file.filename)
        df_todos_paises = pd.read_csv(zf.open(csv_file.filename))
        break

if df_todos_paises.empty:
    exit("Erro ao ler o arquivo 'EdStatsData.csv' que fica dentro do 'Edstats_csv.zip' na pasta 'data'.")
else:
    print("Arquivo 'EdStatsData.csv' carregado com sucesso.")

Abrindo o arquivo EdStatsData.csv
Arquivo 'EdStatsData.csv' carregado com sucesso.


## 3. Separando dados referentes ao Brasil.
O Brasil será o país selecionado, no trabalho de conclusão de curso, como referência para comparações, análises e previsões. Logo, é pertinente que uma faixa temporal seja escolhida de acordo com os seus dados.

In [100]:
df_brasil = df_todos_paises.loc[df_todos_paises['Country Name'].isin(['Brazil'])]

df_todos_paises_lista_colunas = df_brasil.columns.tolist()
ultima_coluna_lixo_massa = df_todos_paises_lista_colunas[len(df_todos_paises_lista_colunas)-1]
df_brasil = df_brasil.drop([ultima_coluna_lixo_massa], axis=1)

df_brasil.index = range(df_brasil.shape[0])

## 4. Selecionar as linhas referentes aos anos de 1998 a 2013 e encontrar os indicadores com mais de 60% de taxa de preenchimento.

In [106]:
for c in df_brasil.columns:
    if(df_brasil[c].dtype == np.float64):
        coluna_numerica = pd.to_numeric(df_brasil[c].name, errors='coerce')
        
        if(coluna_numerica < 1998 or coluna_numerica > 2013):
            del df_brasil[str(coluna_numerica)]

df_massa_selecionada = df_brasil.drop(['Country Code', 'Country Name'], axis = 1)

indicadores_relevantes = []
for indicador in df_massa_selecionada['Indicator Name']:
    df_por_indicador = df_massa_selecionada.loc[df_massa_selecionada['Indicator Name'] == indicador]
    codigo_indicador = df_por_indicador['Indicator Code'].iloc[0]
    df_por_indicador = df_por_indicador.drop(['Indicator Name', 'Indicator Code'], axis = 1)
    media_valores = df_por_indicador.iloc[0].isna().mean()
    if media_valores <= 0.3:
        indicadores_relevantes.append(indicador + ';' + codigo_indicador + ';' + str(1-media_valores))

df_indicadores_relevantes = pd.DataFrame(indicadores_relevantes, columns=['Indicador'], )
df_indicadores_relevantes.to_csv('../data/brasil_relevancia_indicadores.csv', index = False)